In [7]:
import cv2
import numpy as np
import pykinect_azure as pykinect
from pykinect_azure.k4a import calibration
from pykinect_azure.k4a import _k4a

# Initialize the library, if the library is not found, add the library path as an argument
pykinect.initialize_libraries(track_body=True)

# Modify camera configuration
device_config = pykinect.default_configuration
device_config.color_resolution = pykinect.K4A_COLOR_RESOLUTION_1080P
device_config.color_format = pykinect.K4A_IMAGE_FORMAT_COLOR_BGRA32
device_config.camera_fps = pykinect.K4A_FRAMES_PER_SECOND_30
device_config.depth_mode = pykinect.K4A_DEPTH_MODE_WFOV_2X2BINNED
# print(device_config)

# Start device
device = pykinect.start_device(config=device_config)

# Start body tracker
bodyTracker = pykinect.start_body_tracker()

fourcc = cv2.VideoWriter_fourcc(*'H264')  # 'mp4v' is the codec for MP4
out = cv2.VideoWriter('rgbd_video.mp4', fourcc, 30.0, (1920, 1080))  # Adjust the resolution as needed

# clamp 調遠方的點雲 rescaling 近處的
# Set the desired depth range for visualization
min_depth = 0   # Adjust this value based on your specific scenario
max_depth = 100 # Adjust this value based on your specific scenario

# 問題不是FOV 而是distortion，depth不能直接拿來用
# 先去照能照到手的問題
# window_size = (1280, 720)
try:
    while True:
        # Get capture
        capture = device.update()

        # Get body tracker frame
        body_frame = bodyTracker.update()

        # Get the color image
        ret_rgb, img_RGB = capture.get_color_image()
        #不會distortion 的 depth photo
        # ret_depth, img_depth = capture.get_transformed_colored_depth_image()
        ret_depth, img_depth = capture.get_transformed_depth_image()
        
        if not ret_rgb or not ret_depth:
            continue

        normalized_depth = np.clip(img_depth, min_depth, max_depth)
        normalized_depth = (normalized_depth - min_depth) / (max_depth - min_depth) * 255
        normalized_depth = normalized_depth.astype(np.uint8)

        # ret_ir, img_ir = capture.get_ir_image()
        # Display the color image
        cv2.imshow('MIS Kinect View RGB', img_RGB)
        # cv2.resizeWindow('MIS Kinect View RGB', *window_size)
        cv2.imshow('MIS Kinect View D', normalized_depth)
        # cv2.imshow('MIS Kinect View D', img_depth)
        # cv2.imshow('MIS Kinect View IR', img_ir)
        out.write(img_RGB)
        
        # Press q key to stop
        key = cv2.waitKey(1)
        if key == ord('q') or key == 27:  # Check for 'q' key or ESC key
            break

finally:
    # Release the resources
    cv2.destroyAllWindows()
    device.close()
    out.release()